In [26]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('data/places/SK.parquet')

In [3]:
len(df)

94138

In [4]:
df.head()

,id,geometry,bbox,version,update_time,sources,names,categories,confidence,websites,socials,emails,phones,brand,addresses
0,08f8935c9e45406d0321f27c4c0e3195,b'\x00\x00\x00\x00\x01\xc0b\xbdB\x81\xf0/\xa8\...,"{'xmin': -149.9143829345703, 'xmax': -149.9143...",0,2024-06-06T00:00:00.000Z,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'Francúzska Polynézia', 'common': ...",None,0.614162,[https://moanatravel.sk/],[https://www.facebook.com/110918633978989],None,[+421902309850],None,"[{'freeform': 'Trnavská cesta 161/34', 'locali..."
1,08fce8b6b1d969b003f7de71249933f9,b'\x00\x00\x00\x00\x01\xc0RU|\xb7v#s\xc0B\xbcz...,"{'xmin': -73.33573913574219, 'xmax': -73.33573...",0,2024-06-06T00:00:00.000Z,"[{'property': '', 'dataset': 'msft', 'record_i...","{'primary': 'MAROJAN consulting, s. r. o.', 'c...","{'main': 'marketing_consultant', 'alternate': ...",0.770000,[],None,None,[0905847345],None,"[{'freeform': 'P. O. Hviezdoslava 80, 46', 'lo..."
2,08fa81046d025ac003247079c8cfce92,b'\x00\x00\x00\x00\x01\xc0G5X6!\xfa\xfd\xc07\x...,"{'xmin': -46.41675567626953, 'xmax': -46.41674...",0,2024-06-06T00:00:00.000Z,"[{'property': '', 'dataset': 'msft', 'record_i...","{'primary': 'Lekáreň Dr.Max', 'common': None, ...","{'main': 'pharmacy', 'alternate': ['retail']}",0.770000,[https://www.drmax.sk/lekarne/sol-sol-200?y_so...,None,None,[0901961371],None,"[{'freeform': 'Soľ 200', 'locality': 'Soľ', 'p..."
3,08fa810453132a4a03f4f3a6dfd44291,b'\x00\x00\x00\x00\x01\xc0G!\xb2\x98n o\xc07\x...,"{'xmin': -46.26326370239258, 'xmax': -46.26325...",0,2024-06-06T00:00:00.000Z,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': 'ChataAdam', 'common': None, 'rule...","{'main': 'hotel', 'alternate': ['cottage', 'la...",0.490752,[http://www.chataadam.eu],[https://www.facebook.com/714930065266990],None,[+421911269720],None,"[{'freeform': 'Ždiar 168', 'locality': 'Ždiar'..."
4,08fa812d5580615403e4309554041cf6,b'\x00\x00\x00\x00\x01\xc0F\xb1\xc9f\xe6\xbe\x...,"{'xmin': -45.38896560668945, 'xmax': -45.38895...",0,2024-06-06T00:00:00.000Z,"[{'property': '', 'dataset': 'msft', 'record_i...","{'primary': 'Lekáreň Dr.Max', 'common': None, ...","{'main': 'pharmacy', 'alternate': ['retail']}",0.770000,[https://www.drmax.sk/lekarne/martin-m-r-stefa...,None,None,[0901961242],None,"[{'freeform': 'M. R. Štefánika 40', 'locality'..."


In [5]:
mains = df.categories.apply(lambda x: None if x is None else x['main'])

In [6]:
# mains_df = mains.value_counts().reset_index()
# mains_df['is_sports_related'] = 0
# mains_df.to_excel('data/places/categories.xlsx', index=False)

mains_df = pd.read_excel('data/places/categories_CLEAN.xlsx')

In [7]:
mains_df = mains_df[mains_df.is_sports_related == 1].copy()

In [8]:
mains_df.categories

11                              gym
62                       dance_club
67      sports_and_recreation_venue
100                      ski_resort
103                    dance_school
                   ...             
959         adventure_sports_center
978                      salsa_club
1027                   kiteboarding
1029                 cricket_ground
1048                 baseball_field
Name: categories, Length: 64, dtype: object

In [9]:
df = df[mains.isin(mains_df.categories)].copy()

In [10]:
CATEGORIES_MAP = {
    'Fitness': ['gym', 'fitness_trainer', 'gymnastics_center', 'sports_and_fitness_instruction'],
    'Tanec': ['dance_club', 'dance_school', 'salsa_club'],
    'Lyžovanie': ['ski_resort', 'ski_and_snowboard_school'],
    'Plávanie': ['swimming_pool', 'swimming_instructor', 'pool_and_hot_tub_services', 'hot_tubs_and_pools'],
    'Yoga': ['yoga_studio'],
    'Bojové umenie': ['martial_arts_club', 'boxing_class'],
    'Futbal': ['soccer_field', 'soccer_stadium', 'football_stadium'],
    'Bowling': ['bowling_alley'],
    'Tenis': ['tennis_court', 'tennis_stadium'],
    'Pilates': ['pilates_studio'],
    'Jazda na koni': ['horseback_riding_service', 'horse_trainer'],
    'Paintball': ['paintball'],
    'Bicyklovanie': ['bike_rentals', 'mountain_bike_trails', 'cycling_classes'],
    'Skating': ['skate_park'],
    'Hokej': ['ice_skating_rink', 'hockey_arena', 'hockey_field'],
    'Lezenie': ['rock_climbing_spot'],
    'Biliard': ['pool_billiards'],
    'Golf': ['miniature_golf_course', 'golf_instructor'],
    'Motokáry': ['go_kart_club', 'race_track'],
    'Vodné športy': ['scuba_diving_center', 'canoe_and_kayak_hire_service', 'boat_rental_and_training', 'boat_dealer', 'boat_tours', 'paddleboarding_center', 'surfing', 'scuba_diving_instruction', 'diving_center', 'rafting_kayaking_area', 'kiteboarding'],
    'Beh': ['track_stadium'],
    'Basketbal': ['basketball_court', 'basketball_stadium'],
    'Squash': ['squash_court'],
    'Lukostrelectvo': ['archery_range'],
    'Discgolf': ['disc_golf_course'],
    'Volejbal': ['volleyball_court'],
    'Badminton': ['badminton_court'],
    'Baseball': ['cricket_ground', 'baseball_field'],
    'Iné': ['hot_air_balloons_tour', 'sky_diving', 'adventure_sports_center', 'sports_and_recreation_venue'],
}

In [11]:
def get_main_category(categories):
    for category, tags in CATEGORIES_MAP.items():
        if categories['main'] in tags:
            return category
    return None
    

In [12]:
df['main_category'] = df.categories.apply(get_main_category)

In [13]:
from shapely import wkb

def get_coordinates(geometry):
    point = wkb.loads(geometry)
    # The coordinates are switched
    return (point.y, point.x)

In [14]:
df['coordinates'] = df['geometry'].apply(get_coordinates)

In [15]:
df['coordinates']

28        (84.3775551, -81.51684)
98            (49.14281, 9.22268)
260          (47.80477, 17.92035)
261          (47.80477, 17.92035)
312      (47.7658973, 18.0900192)
                   ...           
93867        (49.31229, 21.57539)
93894    (49.3127518, 21.5772495)
93938      (49.3730014, 21.69895)
93951    (49.4259396, 21.6295756)
94050    (49.3067048, 21.9434216)
Name: coordinates, Length: 3725, dtype: object

In [54]:
import re
import requests
import urllib

def is_facebook_number_url(url):
    pattern = r"^https://www\.facebook\.com/(\d+)/?$"
    return bool(re.match(pattern, url))

def parse_url_from_response_url(response_url):
    try:
        return urllib.parse.unquote(response_url.split('next=')[1])
    except:
        return None

def get_redirect_link(url):
            
    try:
        print(f"Getting redirect link for {url}")
        r = requests.get(url, timeout=10)
        r.raise_for_status()

        if not is_facebook_number_url(r.url):
            
            next_url = parse_url_from_response_url(r.url)
            if next_url is None:
                print(r.url)
                return r.url
            elif is_facebook_number_url(next_url): # Sometimes the redirect link is still a number URL
                return None

            print(next_url)
            return next_url
        
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"Error getting redirect link for {url}")
        return None

In [31]:
df['facebook_url'] = df.socials.apply(lambda x: None if not isinstance(x, np.ndarray) else get_redirect_link(x[0]))

Getting redirect link for https://www.facebook.com/106309755598022
https://www.facebook.com/people/Kryt%C3%A1-plav%C3%A1re%C5%88-Kom%C3%A1rno/100086381752580/
Getting redirect link for https://www.facebook.com/592306130972743
https://www.facebook.com/han.trener/
Getting redirect link for https://www.facebook.com/287728514906586
https://www.facebook.com/people/Sportk%C3%B6zpont-Terasz/100063463403942/
Getting redirect link for https://www.facebook.com/869934463117923
https://www.facebook.com/people/Popeye-Gym-Ekel/100066479897332/
Getting redirect link for https://www.facebook.com/247313155664018
https://www.facebook.com/hammer.gym.komarno/
Getting redirect link for https://www.facebook.com/124847515054636
https://www.facebook.com/OMYOGAKOMARNO/
Getting redirect link for https://www.facebook.com/452149314858889
https://www.facebook.com/pages/Klzisko%20Komarno%20Ostrov/452149314858889/
Getting redirect link for https://www.facebook.com/377042139035441
https://www.facebook.com/LifeTilt.sk

In [55]:
for i, row in df[df['facebook_url'].isnull()].iterrows():
    if isinstance(row['socials'], np.ndarray):
        df.at[i, 'facebook_url'] = get_redirect_link(row['socials'][0])

Getting redirect link for https://www.facebook.com/865127563534475
Getting redirect link for https://www.facebook.com/220500844706456
Getting redirect link for https://www.facebook.com/230055566777
Getting redirect link for https://www.facebook.com/115982098438539
Getting redirect link for https://www.facebook.com/312476865535931
Getting redirect link for https://www.facebook.com/305545580095726
Getting redirect link for https://www.facebook.com/99475306371
Getting redirect link for https://www.facebook.com/253187181380411
Getting redirect link for https://www.facebook.com/189799324544685
Getting redirect link for https://www.facebook.com/1719436818384975
Getting redirect link for https://www.facebook.com/134315946634346
Getting redirect link for https://www.facebook.com/364076797131655
Getting redirect link for https://www.facebook.com/212045662172090
Getting redirect link for https://www.facebook.com/227964717739036
Getting redirect link for https://www.facebook.com/208117259221960
G

In [57]:
df.to_parquet('data/places_SK.parquet', index=False)

In [187]:
venues = []
for i, row in df.iterrows():
    venues.append({
        'name': row['names']['primary'],
        'coordinates': row['coordinates'],
        'website': None if row['websites'] is None else row['websites'][0],
        'facebook_url': None if row['socials'] is None else row['socials'][0],
        'email': None if row['emails'] is None else row['emails'][0],
        'phone': None if row['phones'] is None else row['phones'][0],
        'address_freeform': row['addresses'][0]['freeform'],
        'address_locality': row['addresses'][0]['locality'],
        'address_postcode': row['addresses'][0]['postcode'],
        'address_region': row['addresses'][0]['region'],
        'address_country': row['addresses'][0]['country'],
        'main_category': row['main_category'],
        'categories': {
            'main': row['categories']['main'],
            'alternate': None if row['categories']['alternate'] is None else list(row['categories']['alternate'])
        }
    })

In [188]:
import json

In [190]:
with open('data/venues_overture_maps.json', 'w', encoding='utf-8') as f:
    json.dump({'venues': venues}, f, indent=4)

In [186]:
with open('data/categories_map.json', 'w', encoding='utf-8') as f:
    json.dump(CATEGORIES_MAP, f, indent=4)

In [155]:
sum(df.addresses.isnull())

0

In [161]:
for n in df.categories.sample(100):
    print(n)

{'main': 'dance_club', 'alternate': array(['bar', 'arts_and_entertainment'], dtype=object)}
{'main': 'gym', 'alternate': array(['life_coach', 'fitness_trainer'], dtype=object)}
{'main': 'bowling_alley', 'alternate': array(['pizza_restaurant', 'restaurant'], dtype=object)}
{'main': 'sports_and_recreation_venue', 'alternate': array(['amusement_park', 'active_life'], dtype=object)}
{'main': 'sports_and_recreation_venue', 'alternate': array(['sports_club_and_league', 'active_life'], dtype=object)}
{'main': 'fitness_trainer', 'alternate': None}
{'main': 'sports_and_recreation_venue', 'alternate': array(['travel', 'accommodation'], dtype=object)}
{'main': 'tennis_court', 'alternate': array(['thai_restaurant'], dtype=object)}
{'main': 'paintball', 'alternate': None}
{'main': 'gym', 'alternate': array(['sports_club_and_league', 'stadium_arena'], dtype=object)}
{'main': 'paddleboarding_center', 'alternate': array(['active_life', 'sports_club_and_league'], dtype=object)}
{'main': 'bowling_alley'

In [164]:
df.categories.iloc[0]

{'main': 'swimming_pool', 'alternate': None}